<a href="https://colab.research.google.com/github/miki727/ABCNY_Project/blob/main/Machine%20learning/Shuning%20Ma/ml_model_binary_score_piml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [2]:
!pip install jellyfish
!pip install fuzzywuzzy
!pip install PiML

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import time
import re
import pandas as pd
import numpy as np
import jellyfish as jf
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [4]:
df = pd.read_csv('ml_dataset_score.csv')
df

,Original Name,Test Case Name,Test Type,Label,Bridger Score
0,JAM'YAH TA'AWUN AL-ISLAMIA,JAM'YAH TA'AWUN AL-ISLAMIA,Fuzzy Match,Match,100
1,"GLOBOVISION TELE CA, CORP.","GLOBOVISION TELE CA, CORP.",Fuzzy Match,Match,100
2,KOREAN PEOPLE'S ARMY,KOREAN PEOPLE'S ARMY,Fuzzy Match,Match,100
3,RANCHO LA HERRADURA,RANCHO LA HERRADURA,Fuzzy Match,Match,100
4,"TECHNO ENERGY, S.A.","TECHNO ENERGY, S.A.",Fuzzy Match,Match,100
...,...,...,...,...,...
18503,GLOBAL AGE LIMITED,"SHAABAN, Bouthaina",Non-Match,Not Match,0
18504,OBSHCHESTVO S OGRANICHENNOI OTVETSTVENNOSTYU M...,TAMIN KALAYE SABZ ARAS COMPANY,Non-Match,Not Match,0
18505,"AHMED, Qassim Abdullah Ali",KOREA HAEGUMGANG TRADING CORPORATION,Non-Match,Not Match,0
18506,JINGHO TECHNOLOGY CO. LIMITED,COMITE' DE BIENFAISANCE ET DE SECOURS AUX PALE...,Non-Match,Not Match,0


# Feature engineering

In [5]:
def matching_numbers(original_name, test_case_name):

    original_numbers = set(re.findall(r'[0-9]+', original_name))
    test_case_numbers = set(re.findall(r'[0-9]+', test_case_name))    
    union = original_numbers.union(test_case_numbers)
    intersection = original_numbers.intersection(test_case_numbers)

    if len(original_numbers)==0 and len(test_case_numbers) == 0:
        return 1
    else:
        return (len(intersection)/ len(union))

In [6]:
def engineer_features(df):

    df['Test Case Name'] = df['Test Case Name'].str.lower()
    df['Original Name'] = df['Original Name'].str.lower()

    df['levenshtein_distance'] = df.apply(
    lambda x: jf.levenshtein_distance(x['Original Name'], 
                                      x['Test Case Name']), axis=1)

    df['damerau_levenshtein_distance'] = df.apply(
    lambda x: jf.damerau_levenshtein_distance(x['Original Name'], 
                                              x['Test Case Name']), axis=1)

    df['hamming_distance'] = df.apply(
    lambda x: jf.hamming_distance(x['Original Name'], 
                                  x['Test Case Name']), axis=1)

    df['jaro_similarity'] = df.apply(
    lambda x: jf.jaro_similarity(x['Original Name'], 
                                  x['Test Case Name']), axis=1)

    df['jaro_winkler_similarity'] = df.apply(
    lambda x: jf.jaro_winkler_similarity(x['Original Name'], 
                                         x['Test Case Name']), axis=1)

    df['match_rating_comparison'] = df.apply(
    lambda x: jf.match_rating_comparison(x['Original Name'], 
                                         x['Test Case Name']), axis=1).fillna(0).astype(int)

    df['ratio'] = df.apply(
    lambda x: fuzz.ratio(x['Original Name'], 
                         x['Test Case Name']), axis=1)

    df['partial_ratio'] = df.apply(
    lambda x: fuzz.partial_ratio(x['Original Name'], 
                                 x['Test Case Name']), axis=1)

    df['token_sort_ratio'] = df.apply(
    lambda x: fuzz.token_sort_ratio(x['Original Name'], 
                                    x['Test Case Name']), axis=1)

    df['token_set_ratio'] = df.apply(
    lambda x: fuzz.token_set_ratio(x['Original Name'], 
                                   x['Test Case Name']), axis=1)

    df['w_ratio'] = df.apply(
    lambda x: fuzz.WRatio(x['Original Name'], 
                          x['Test Case Name']), axis=1)

    df['uq_ratio'] = df.apply(
    lambda x: fuzz.UQRatio(x['Original Name'], 
                          x['Test Case Name']), axis=1)

    df['q_ratio'] = df.apply(
    lambda x: fuzz.QRatio(x['Original Name'], 
                          x['Test Case Name']), axis=1)    

    df['matching_numbers'] = df.apply(
    lambda x: matching_numbers(x['Original Name'], 
                               x['Test Case Name']), axis=1)

    df['matching_numbers_log'] = (df['matching_numbers']+1).apply(np.log)

    df['log_fuzz_score'] = (df['ratio'] + df['partial_ratio'] + 
                            df['token_sort_ratio'] + df['token_set_ratio']).apply(np.log)

    df['log_fuzz_score_numbers'] = df['log_fuzz_score'] + (df['matching_numbers']).apply(np.log)

    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.fillna(value=0, inplace=True)

    return df

In [7]:
df = engineer_features(df)
df

,Original Name,Test Case Name,Test Type,Label,Bridger Score,levenshtein_distance,damerau_levenshtein_distance,hamming_distance,jaro_similarity,jaro_winkler_similarity,...,partial_ratio,token_sort_ratio,token_set_ratio,w_ratio,uq_ratio,q_ratio,matching_numbers,matching_numbers_log,log_fuzz_score,log_fuzz_score_numbers
0,jam'yah ta'awun al-islamia,jam'yah ta'awun al-islamia,Fuzzy Match,Match,100,0,0,0,1.000000,1.000000,...,100,100,100,100,100,100,1.0,0.693147,5.991465,5.991465
1,"globovision tele ca, corp.","globovision tele ca, corp.",Fuzzy Match,Match,100,0,0,0,1.000000,1.000000,...,100,100,100,100,100,100,1.0,0.693147,5.991465,5.991465
2,korean people's army,korean people's army,Fuzzy Match,Match,100,0,0,0,1.000000,1.000000,...,100,100,100,100,100,100,1.0,0.693147,5.991465,5.991465
3,rancho la herradura,rancho la herradura,Fuzzy Match,Match,100,0,0,0,1.000000,1.000000,...,100,100,100,100,100,100,1.0,0.693147,5.991465,5.991465
4,"techno energy, s.a.","techno energy, s.a.",Fuzzy Match,Match,100,0,0,0,1.000000,1.000000,...,100,100,100,100,100,100,1.0,0.693147,5.991465,5.991465
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18503,global age limited,"shaaban, bouthaina",Non-Match,Not Match,0,16,16,18,0.449735,0.449735,...,30,23,23,28,28,28,1.0,0.693147,4.644391,4.644391
18504,obshchestvo s ogranichennoi otvetstvennostyu m...,tamin kalaye sabz aras company,Non-Match,Not Match,0,44,44,51,0.510733,0.510733,...,27,25,25,28,20,20,1.0,0.693147,4.574711,4.574711
18505,"ahmed, qassim abdullah ali",korea haegumgang trading corporation,Non-Match,Not Match,0,28,28,35,0.553419,0.553419,...,23,10,10,23,23,23,1.0,0.693147,4.189655,4.189655
18506,jingho technology co. limited,comite' de bienfaisance et de secours aux pale...,Non-Match,Not Match,0,43,43,54,0.546201,0.546201,...,24,30,31,33,17,17,1.0,0.693147,4.624973,4.624973


In [8]:
df = df.drop(index = df[df['Bridger Score'] == 'ERROR WITH TEST CASE'].index.tolist())
for i in list(df[df['Bridger Score'] == 'NO MATCH'].index):
    df.loc[i, 'Bridger Score'] = np.random.randint(50, 70)
for i in list(df[df['Bridger Score'] == '0'].index):
    df.loc[i, 'Bridger Score'] = np.random.randint(10, 50)
df

,Original Name,Test Case Name,Test Type,Label,Bridger Score,levenshtein_distance,damerau_levenshtein_distance,hamming_distance,jaro_similarity,jaro_winkler_similarity,...,partial_ratio,token_sort_ratio,token_set_ratio,w_ratio,uq_ratio,q_ratio,matching_numbers,matching_numbers_log,log_fuzz_score,log_fuzz_score_numbers
0,jam'yah ta'awun al-islamia,jam'yah ta'awun al-islamia,Fuzzy Match,Match,100,0,0,0,1.000000,1.000000,...,100,100,100,100,100,100,1.0,0.693147,5.991465,5.991465
1,"globovision tele ca, corp.","globovision tele ca, corp.",Fuzzy Match,Match,100,0,0,0,1.000000,1.000000,...,100,100,100,100,100,100,1.0,0.693147,5.991465,5.991465
2,korean people's army,korean people's army,Fuzzy Match,Match,100,0,0,0,1.000000,1.000000,...,100,100,100,100,100,100,1.0,0.693147,5.991465,5.991465
3,rancho la herradura,rancho la herradura,Fuzzy Match,Match,100,0,0,0,1.000000,1.000000,...,100,100,100,100,100,100,1.0,0.693147,5.991465,5.991465
4,"techno energy, s.a.","techno energy, s.a.",Fuzzy Match,Match,100,0,0,0,1.000000,1.000000,...,100,100,100,100,100,100,1.0,0.693147,5.991465,5.991465
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18503,global age limited,"shaaban, bouthaina",Non-Match,Not Match,36,16,16,18,0.449735,0.449735,...,30,23,23,28,28,28,1.0,0.693147,4.644391,4.644391
18504,obshchestvo s ogranichennoi otvetstvennostyu m...,tamin kalaye sabz aras company,Non-Match,Not Match,21,44,44,51,0.510733,0.510733,...,27,25,25,28,20,20,1.0,0.693147,4.574711,4.574711
18505,"ahmed, qassim abdullah ali",korea haegumgang trading corporation,Non-Match,Not Match,24,28,28,35,0.553419,0.553419,...,23,10,10,23,23,23,1.0,0.693147,4.189655,4.189655
18506,jingho technology co. limited,comite' de bienfaisance et de secours aux pale...,Non-Match,Not Match,25,43,43,54,0.546201,0.546201,...,24,30,31,33,17,17,1.0,0.693147,4.624973,4.624973


In [9]:
df['Label'] = df['Label'].map({'Match' : 1, 'Not Match' : 0})
df

,Original Name,Test Case Name,Test Type,Label,Bridger Score,levenshtein_distance,damerau_levenshtein_distance,hamming_distance,jaro_similarity,jaro_winkler_similarity,...,partial_ratio,token_sort_ratio,token_set_ratio,w_ratio,uq_ratio,q_ratio,matching_numbers,matching_numbers_log,log_fuzz_score,log_fuzz_score_numbers
0,jam'yah ta'awun al-islamia,jam'yah ta'awun al-islamia,Fuzzy Match,1,100,0,0,0,1.000000,1.000000,...,100,100,100,100,100,100,1.0,0.693147,5.991465,5.991465
1,"globovision tele ca, corp.","globovision tele ca, corp.",Fuzzy Match,1,100,0,0,0,1.000000,1.000000,...,100,100,100,100,100,100,1.0,0.693147,5.991465,5.991465
2,korean people's army,korean people's army,Fuzzy Match,1,100,0,0,0,1.000000,1.000000,...,100,100,100,100,100,100,1.0,0.693147,5.991465,5.991465
3,rancho la herradura,rancho la herradura,Fuzzy Match,1,100,0,0,0,1.000000,1.000000,...,100,100,100,100,100,100,1.0,0.693147,5.991465,5.991465
4,"techno energy, s.a.","techno energy, s.a.",Fuzzy Match,1,100,0,0,0,1.000000,1.000000,...,100,100,100,100,100,100,1.0,0.693147,5.991465,5.991465
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18503,global age limited,"shaaban, bouthaina",Non-Match,0,36,16,16,18,0.449735,0.449735,...,30,23,23,28,28,28,1.0,0.693147,4.644391,4.644391
18504,obshchestvo s ogranichennoi otvetstvennostyu m...,tamin kalaye sabz aras company,Non-Match,0,21,44,44,51,0.510733,0.510733,...,27,25,25,28,20,20,1.0,0.693147,4.574711,4.574711
18505,"ahmed, qassim abdullah ali",korea haegumgang trading corporation,Non-Match,0,24,28,28,35,0.553419,0.553419,...,23,10,10,23,23,23,1.0,0.693147,4.189655,4.189655
18506,jingho technology co. limited,comite' de bienfaisance et de secours aux pale...,Non-Match,0,25,43,43,54,0.546201,0.546201,...,24,30,31,33,17,17,1.0,0.693147,4.624973,4.624973


# Select the model

In [10]:
from piml import Experiment
exp = Experiment()

In [11]:
exp.data_loader(data=df)

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

,Original Name,Test Case Name,Test Type,Label,Bridger Score,levenshtein_distance,damerau_levenshtein_distance,hamming_distance,jaro_similarity,jaro_winkler_similarity,match_rating_comparison,ratio,partial_ratio,token_sort_ratio,token_set_ratio,w_ratio,uq_ratio,q_ratio,matching_numbers,matching_numbers_log,log_fuzz_score,log_fuzz_score_numbers
0,jam'yah ta'awun al-islamia,jam'yah ta'awun al-islamia,Fuzzy Match,1.0,100.0,0.0,0.0,0.0,1.0,1.0,1.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,1.0,0.693147,5.991465,5.991465
1,"globovision tele ca, corp.","globovision tele ca, corp.",Fuzzy Match,1.0,100.0,0.0,0.0,0.0,1.0,1.0,1.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,1.0,0.693147,5.991465,5.991465
2,korean people's army,korean people's army,Fuzzy Match,1.0,100.0,0.0,0.0,0.0,1.0,1.0,1.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,1.0,0.693147,5.991465,5.991465
3,rancho la herradura,rancho la herradura,Fuzzy Match,1.0,100.0,0.0,0.0,0.0,1.0,1.0,1.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,1.0,0.693147,5.991465,5.991465
4,"techno energy, s.a.","techno energy, s.a.",Fuzzy Match,1.0,100.0,0.0,0.0,0.0,1.0,1.0,1.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,1.0,0.693147,5.991465,5.991465
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18501,global age limited,"shaaban, bouthaina",Non-Match,0.0,36.0,16.0,16.0,18.0,0.449735,0.449735,0.0,28.0,30.0,23.0,23.0,28.0,28.0,28.0,1.0,0.693147,4.644391,4.644391
18502,obshchestvo s ogranichennoi otvetstvennostyu m...,tamin kalaye sabz aras company,Non-Match,0.0,21.0,44.0,44.0,51.0,0.510733,0.510733,0.0,20.0,27.0,25.0,25.0,28.0,20.0,20.0,1.0,0.693147,4.574711,4.574711
18503,"ahmed, qassim abdullah ali",korea haegumgang trading corporation,Non-Match,0.0,24.0,28.0,28.0,35.0,0.553419,0.553419,0.0,23.0,23.0,10.0,10.0,23.0,23.0,23.0,1.0,0.693147,4.189655,4.189655
18504,jingho technology co. limited,comite' de bienfaisance et de secours aux pale...,Non-Match,0.0,25.0,43.0,43.0,54.0,0.546201,0.546201,0.0,17.0,24.0,30.0,31.0,33.0,17.0,17.0,1.0,0.693147,4.624973,4.624973


In [12]:
exp.data_summary(feature_exclude=["Original Name", "Test Case Name", "Test Type"])

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

HTML(value='Data Shape:(18506, 22)')

In [13]:
exp.data_prepare(target='Label', task_type='Classification', test_ratio=0.3, random_state=0)

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

,0,1
0,Excluded columns,"[Original Name, Test Case Name, Test Type]"
1,Target variable,Label
2,Sample weight,None
3,Task type,Classification
4,Split method,Random
5,Test ratio,0.3
6,Random state,0


In [14]:
# exp.feature_select(threshold=0.98)

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

In [15]:
exp.eda()

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

<IPython.core.display.Javascript object>

In [16]:
## ReLU-DNN: ReLU Deep Neural Networks with L1-Regularization, e.g. L1_reg = 0.0008
from piml.models import ReluDNNClassifier

clf1 = ReluDNNClassifier(hidden_layer_sizes=(40, 40), l1_reg=0.0008,
                        batch_size=500, learning_rate=0.001)
exp.model_train(model=clf1, name='ReLU-DNN')
exp.model_diagnose(model="ReLU-DNN", show='accuracy_result')

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

HTML(value="<p class='notification info'>Register ReLU-DNN Done</p>")

,ACC,AUC,Recall,Precision,F1
,,,,,
Train,0.9993,1.0000,0.9988,0.9998,0.9993
Test,0.9993,1.0000,0.9989,0.9996,0.9993
Gap,-0.0000,-0.0000,0.0001,-0.0002,-0.0000


In [17]:
## GAMI-Net: GAM-Neural Networks with Structured Interactions
from piml.models import GAMINetClassifier

clf2 = GAMINetClassifier(interact_num=10, loss_threshold=0.01,  
                        subnet_size_main_effect=[20], 
                        subnet_size_interaction=[20,20])
exp.model_train(model=clf2, name='GAMI-Net')
exp.model_diagnose(model="GAMI-Net", show='accuracy_result')

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

HTML(value="<p class='notification info'>Register GAMI-Net Done</p>")

,ACC,AUC,Recall,Precision,F1
,,,,,
Train,1.0000,1.0000,1.0000,1.0000,1.0000
Test,0.9998,1.0000,1.0000,0.9996,0.9998
Gap,-0.0002,-0.0000,0.0000,-0.0004,-0.0002


In [18]:
## EBM: Explainable Boosting Machine from MS Research
from piml.models import ExplainableBoostingClassifier

clf3 = ExplainableBoostingClassifier(interactions=10, random_state=0)
exp.model_train(model=clf3, name='EBM')
exp.model_diagnose(model="EBM", show='accuracy_result')

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

HTML(value="<p class='notification info'>Register EBM Done</p>")

,ACC,AUC,Recall,Precision,F1
,,,,,
Train,1.0000,1.0000,1.0000,1.0000,1.0000
Test,1.0000,1.0000,1.0000,1.0000,1.0000
Gap,0.0000,0.0000,0.0000,0.0000,0.0000


In [19]:
exp.model_compare(models=['ReLU-DNN', 'GAMI-Net', 'EBM'], show='accuracy')

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

<IPython.core.display.Javascript object>

In [20]:
exp.model_explain()

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

<IPython.core.display.Javascript object>

# Run test data

In [21]:
df_test = pd.read_csv('New_ML_Data.csv')
df_test

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

,Bridger Score,levenshtein_distance,damerau_levenshtein_distance,hamming_distance,jaro_similarity,jaro_winkler_similarity,match_rating_comparison,ratio,partial_ratio,token_sort_ratio,token_set_ratio,w_ratio,uq_ratio,q_ratio,matching_numbers,matching_numbers_log,log_fuzz_score,log_fuzz_score_numbers,Disposition
0,92,1,1,2,0.972222,0.983333,1,96,91,96,96,96,96,96,1,0.693147,5.937536,5.937536,No Match
1,92,10,10,14,0.659722,0.659722,0,43,50,74,80,76,44,44,1,0.693147,5.509388,5.509388,No Match
2,92,13,13,16,0.586971,0.586971,0,40,36,55,73,69,41,41,1,0.693147,5.318120,5.318120,No Match
3,92,11,11,16,0.603454,0.603454,0,41,38,79,79,75,50,50,1,0.693147,5.468060,5.468060,No Match
4,95,6,6,15,0.760317,0.760317,1,69,74,55,73,69,69,69,1,0.693147,5.602119,5.602119,No Match
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73855,94,37,37,44,0.605797,0.605797,0,32,70,36,100,86,32,32,1,0.693147,5.472271,5.472271,No Match
73856,100,0,0,0,1.000000,1.000000,1,100,100,100,100,100,100,100,1,0.693147,5.991465,5.991465,No Match
73857,100,0,0,0,1.000000,1.000000,1,100,100,100,100,100,100,100,1,0.693147,5.991465,5.991465,Match
73858,100,0,0,0,1.000000,1.000000,1,100,100,100,100,100,100,100,1,0.693147,5.991465,5.991465,No Match


In [22]:
df_test['Disposition'] = df_test['Disposition'].map({'Match' : 1, 'No Match' : 0})

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

In [23]:
exp.experiment.registered_pipeline

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

{'ReLU-DNN': <piml.workflow.pipeline.ModelPipeline at 0x7f8db3562090>,
 'GAMI-Net': <piml.workflow.pipeline.ModelPipeline at 0x7f8db1c34e50>,
 'EBM': <piml.workflow.pipeline.ModelPipeline at 0x7f8db1c34c50>}

In [24]:
df_test.iloc[:, :-1] = df_test.iloc[:, :-1].astype(object)
df_test.iloc[:, :-1].info()

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73860 entries, 0 to 73859
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Bridger Score                 73860 non-null  object
 1   levenshtein_distance          73860 non-null  object
 2   damerau_levenshtein_distance  73860 non-null  object
 3   hamming_distance              73860 non-null  object
 4   jaro_similarity               73860 non-null  object
 5   jaro_winkler_similarity       73860 non-null  object
 6   match_rating_comparison       73860 non-null  object
 7   ratio                         73860 non-null  object
 8   partial_ratio                 73860 non-null  object
 9   token_sort_ratio              73860 non-null  object
 10  token_set_ratio               73860 non-null  object
 11  w_ratio                       73860 non-null  object
 12  uq_ratio                      73860 non-null  object
 13  q_ratio         

## ReLU-DNN

In [26]:
y_pred_ReLU = pd.DataFrame(exp.experiment.registered_pipeline['ReLU-DNN'].predict_proba(df_test.iloc[:, :-1].values))
y_pred_ReLU = pd.DataFrame(y_pred_ReLU[1])
y_pred_ReLU.columns= ['prob']
y_pred_ReLU['label'] = np.where(y_pred_ReLU['prob'] >= 0.9, 1, 0)
y_pred_ReLU

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

,prob,label
0,1.0,1
1,1.0,1
2,1.0,1
3,1.0,1
4,1.0,1
...,...,...
73855,1.0,1
73856,1.0,1
73857,1.0,1
73858,1.0,1


In [27]:
y_pred = y_pred_ReLU['label'].values
y_test = df_test['Disposition'].values
print(classification_report(y_test, y_pred, labels=[1, 0], 
                            target_names=['match', 'not match']))

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

              precision    recall  f1-score   support

       match       0.04      1.00      0.08      3047
   not match       0.00      0.00      0.00     70813

    accuracy                           0.04     73860
   macro avg       0.02      0.50      0.04     73860
weighted avg       0.00      0.04      0.00     73860



In [28]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
print('tp: {}\nfp: {}\nfn: {}\ntn: {}'.format(tp, fp, fn, tn))

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

tp: 3047
fp: 70813
fn: 0
tn: 0


## GAMI-Net

In [29]:
y_pred_GAMI = pd.DataFrame(exp.experiment.registered_pipeline['GAMI-Net'].predict_proba(df_test.iloc[:, :-1].values))
y_pred_GAMI = pd.DataFrame(y_pred_GAMI[1])
y_pred_GAMI.columns= ['prob']
y_pred_GAMI['label'] = np.where(y_pred_GAMI['prob'] >= 0.9, 1, 0)
y_pred_GAMI

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

,prob,label
0,1.0,1
1,1.0,1
2,1.0,1
3,1.0,1
4,1.0,1
...,...,...
73855,1.0,1
73856,1.0,1
73857,1.0,1
73858,1.0,1


In [30]:
y_pred = y_pred_GAMI['label'].values
y_test = df_test['Disposition'].values
print(classification_report(y_test, y_pred, labels=[1, 0], 
                            target_names=['match', 'not match']))

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

              precision    recall  f1-score   support

       match       0.04      1.00      0.08      3047
   not match       0.00      0.00      0.00     70813

    accuracy                           0.04     73860
   macro avg       0.02      0.50      0.04     73860
weighted avg       0.00      0.04      0.00     73860



In [31]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
print('tp: {}\nfp: {}\nfn: {}\ntn: {}'.format(tp, fp, fn, tn))

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

tp: 3047
fp: 70813
fn: 0
tn: 0


## EBM

In [32]:
y_pred_EBM = pd.DataFrame(exp.experiment.registered_pipeline['EBM'].predict_proba(df_test.iloc[:, :-1].values))
y_pred_EBM = pd.DataFrame(y_pred_EBM[1])
y_pred_EBM.columns= ['prob']
y_pred_EBM['label'] = np.where(y_pred_EBM['prob'] >= 0.9, 1, 0)
y_pred_EBM

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

,prob,label
0,1.000000,1
1,0.999998,1
2,0.999980,1
3,0.999999,1
4,0.999999,1
...,...,...
73855,0.999980,1
73856,1.000000,1
73857,1.000000,1
73858,1.000000,1


In [33]:
y_pred = y_pred_EBM['label'].values
y_test = df_test['Disposition'].values
print(classification_report(y_test, y_pred, labels=[1, 0], 
                            target_names=['match', 'not match']))

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

              precision    recall  f1-score   support

       match       0.04      1.00      0.08      3047
   not match       0.00      0.00      0.00     70813

    accuracy                           0.04     73860
   macro avg       0.02      0.50      0.04     73860
weighted avg       0.00      0.04      0.00     73860



In [34]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
print('tp: {}\nfp: {}\nfn: {}\ntn: {}'.format(tp, fp, fn, tn))

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

tp: 3047
fp: 70813
fn: 0
tn: 0
